Data analysis and sales report

In [1]:
# Task 1: Establish a connection 

print("-----Task 1-----")

from mysql.connector.pooling import  MySQLConnectionPool
from mysql.connector import Error
import mysql.connector as connector
 
dbconfig={"database": "little_lemon_db", "user": "root", "password": ""}

try:
    pool_b = MySQLConnectionPool(pool_name = "pool_b", pool_size = 2, **dbconfig)
except Error as er:
    print("Error!")
    print("Code: ", er.errno)
    print("Message: ", er.msg)

print("Pool created!")

# Task 2: Three guests are trying to book dinner slots simultaneously
# Get the connections from pool_b and insert the data in the Bookings table

print("-----Task 2-----")

quests = {
    'quest1' : {'table_number': 8, 'first_name': 'Anees', 'last_name': 'Java', 'booking_time': '18:00', 'employee_id': 6},
    'quest2' : {'table_number': 5, 'first_name': 'Bald', 'last_name': 'Vin', 'booking_time': '19:00', 'employee_id': 6},
    'quest3' : {'table_number': 12, 'first_name': 'Jay', 'last_name': 'Kon', 'booking_time': '19:30', 'employee_id': 6}
}
insert_query = """
INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES ({}, '{}', '{}', '{}', {})
"""

connections = {}

for quest, data in quests.items():
    try:
        connections[quest] = pool_b.get_connection()
        cursor = connections[quest].cursor()
        print("Connection established by " + quest)
        cursor.execute(insert_query.format(data['table_number'],
                                    data['first_name'],
                                    data['last_name'],
                                    data['booking_time'],
                                    data['employee_id']))
        connections[quest].commit()
    except Error as er:
        print("Error getting connection - " + quest + ". Message: " + er.msg + ". Creating new connection!")
        new_conn = connector.connect(user = 'root', password = '')
        pool_b.add_connection(cnx = new_conn)
        connections[quest] = pool_b.get_connection()
        cursor = connections[quest].cursor()
        cursor.execute(insert_query.format(data['table_number'],
                                    data['first_name'],
                                    data['last_name'],
                                    data['booking_time'],
                                    data['employee_id']))
        connections[quest].commit()
    print("Row inserted by " + quest)
for quest, conn in connections.items():
    try:
        conn.close()
        print("Connection closed by " + quest)
    except Error as er:
        print("Error returning connection - " + quest + ". Message: " + er.msg)

# Task 3: Create a report
        
print("-----Task 3-----")
print("-----Report-----")
report = {}

conn = pool_b.get_connection()
cursor = conn.cursor()

# The name and EmployeeID of the Little Lemon manager.
query = "SELECT EmployeeID FROM Employees WHERE Role = 'Manager';"
cursor.execute(query)
result = cursor.fetchall()
report["id_of_manager"] = result[0][0] 

# The name and role of the employee who receives the highest salary.
query = "SELECT Name, Role FROM Employees WHERE Annual_Salary = (SELECT MAX(Annual_Salary) FROM Employees) LIMIT 1;"
cursor.execute(query)
result = cursor.fetchall()
report["employee_with_max_salary"] = result 

# The number of guests booked between 18:00 and 20:00.
query = "SELECT COUNT(*) FROM Bookings WHERE BookingSlot BETWEEN '18:00' AND '20:00';"
cursor.execute(query)
result = cursor.fetchall()
report["quests_between_18_and_20"] = result[0][0]

# The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.
query = "SELECT b.BookingID, CONCAT(b.GuestFirstName, ' ', b.GuestLastName) as FullName FROM Bookings AS b INNER JOIN Employees as e ON b.EmployeeID = e.EmployeeID WHERE e.Role = 'Receptionist' ORDER BY BookingSlot ASC;"
cursor.execute(query)
result = cursor.fetchall()
report["quests_to_be_seated_with_receptionist"] = result

for report_key, report_value in report.items():
    print(str(report_key) + ": " + str(report_value))

conn.close()

# Task 4: Create a stored procedure named BasicSalesReport

print("-----Task 4-----")

conn = pool_b.get_connection()
cursor = conn.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport;")
create_peakhours = """
    CREATE PROCEDURE BasicSalesReport() 
    BEGIN 
        SELECT 
            SUM(BillAmount) AS TotalSales, 
            AVG(BillAmount) AS AvarageSale,
            MIN(BillAmount) AS MinBill,
            MAX(BillAmount) AS MaxBill
        FROM Orders;
    END 
"""
cursor.execute(create_peakhours)
cursor.callproc('BasicSalesReport')

results = next(cursor.stored_results())
dataset = results.fetchall()
column_names = [col[0] for col in results.description]
print(column_names)
for row in dataset:
    print(row)

conn.close()

# Task 5: Display the next three upcoming bookings from the Bookings table

print("-----Task 5-----")
conn = pool_b.get_connection()
cursor = conn.cursor(buffered = True, dictionary = True)

query = "SELECT b.BookingSlot as Slot, CONCAT(b.GuestFirstName, ' ', b.GuestLastName) as FullName, e.Name as EmployeeName, e.Role as EmployeeRole FROM Bookings AS b INNER JOIN Employees as e ON b.EmployeeID = e.EmployeeID ORDER BY Slot ASC;"
cursor.execute(query)
results = cursor.fetchmany(size = 3)
for booking in results:
    print("Booking Slot: " + str(booking['Slot']))
    print("Guest Name: " + booking['FullName'])
    print("Assigned to: " + booking['EmployeeName'] + " (" + booking['EmployeeRole'] + ")")

conn.close()


-----Task 1-----
Pool created!
-----Task 2-----
Connection established by quest1
Row inserted by quest1
Connection established by quest2
Row inserted by quest2
Error getting connection - quest3. Message: Failed getting connection; pool exhausted. Creating new connection!
Row inserted by quest3
Connection closed by quest1
Connection closed by quest2
Error returning connection - quest3. Message: Failed adding connection; queue is full
-----Task 3-----
-----Report-----
id_of_manager: 1
employee_with_max_salary: [('Mario Gollini', 'Manager')]
quests_between_18_and_20: 7
quests_to_be_seated_with_receptionist: [(7, 'Anees Java'), (8, 'Bald Vin'), (9, 'Jay Kon')]
-----Task 4-----
['TotalSales', 'AvarageSale', 'MinBill', 'MaxBill']
(Decimal('243'), Decimal('48.6000'), 37, 86)
-----Task 5-----
Booking Slot: 15:00:00
Guest Name: Vanessa McCarthy
Assigned to: Giorgos Dioudis (Head Chef)
Booking Slot: 17:30:00
Guest Name: Marcos Romero
Assigned to: Fatma Kaya (Assistant Chef)
Booking Slot: 18:00:0